In [10]:
%load_ext autoreload
# Always reload modules so that as you change code in src, it gets loaded
%autoreload 2

import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random
import math
import EoN
import seaborn as sns
import time

# New Imports
#from ctrace.contact_tracing import *
#from ctrace.constraint import *
#from ctrace.solve import *
#from ctrace.simulation import *
#from ctrace.restricted import *
from ctrace.simulation import *
from ctrace.dataset import *
from ctrace.recommender import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
G = load_graph("montgomery")

In [3]:
I = set()
with open(PROJECT_ROOT / "data" / "SIR_Cache" / "t7.json", 'r') as infile:
    j = json.load(infile)
    (S, infected_queue, R) = (j["S"], j["I_Queue"], j["R"])

    # Make infected_queue a list of sets
    infected_queue = [set(s) for s in infected_queue]
    I = I.union(*infected_queue)
    I = list(I)

In [13]:
state = SimulationState(G, (S,I,R), (S,I,R), 2000, .078,1,1)
agent = rand

In [ ]:
while len(state.SIR_known.SIR[1]) != 0:
    print(len(state.SIR_known.SIR[1]))
    to_quarantine = agent(state)
    state.step(to_quarantine)

print(len(state.SIR_known.SIR[1]))